In [1]:
from __future__ import print_function, division

In [2]:
% matplotlib inline

UsageError: Line magic function `%` not found.


In [3]:
import sys, glob, subprocess
from itertools import product

import shutil

import numpy as np
import cv2
import dlib
from PIL import Image
from skimage import io
from matplotlib.gridspec import GridSpec
from matplotlib import pyplot as plt

import tensorflow as tf

import keras
from keras.layers import Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications.resnet50 import preprocess_input, decode_predictions

Using TensorFlow backend.


In [4]:
batch_size=32

In [5]:
# keras 에서 제공하는 data 전처리 작업
datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        #data_format="channels_last",
        preprocessing_function=preprocess_input,
)

In [6]:
train_dir,test_dir=('data/dataset_B_FacialImages/train', 'data/dataset_B_FacialImages/test') # 이미지 데이터 저장경로

In [7]:
# 학습 data 전처리 작업
train_generator = datagen.flow_from_directory(
        train_dir,  #data/dataset_B_FacialImages/train
        target_size=(224, 224),  #모든 이미지를 (224,224)로 사이즈 설정
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
)

Found 2423 images belonging to 2 classes.


In [8]:
# TEST DATA 전처리 작업
validation_generator = datagen.flow_from_directory(
        test_dir, #data/dataset_B_FacialImages/test
        target_size=(224, 224),  #모든 이미지를 (224,224)로 사이즈 설정
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
)

Found 263 images belonging to 2 classes.


In [9]:
# imagenet가중치를 사용한 RESNET 모델
resnet50=keras.applications.resnet50.ResNet50(include_top=True,
                                     weights="imagenet", 
                                     input_tensor=None, 
                                     input_shape=None,
                                     pooling=None, 
                                     )

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [10]:
resnet50.summary() #RESNET MODEL LAYERS

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [11]:
out=Dense(2, activation="softmax")(resnet50.layers[-2].output) #새로운 출력 계층
model_facex=Model(resnet50.input, out, name="EyeInTheSky") # new model with above output layer instead of 1000d from imagenet
model_facex.compile(loss="categorical_crossentropy",optimizer='rmsprop',metrics=['accuracy'])
#A모든 계층이 학습 가능하다.

In [12]:
model_facex.summary()

Model: "EyeInTheSky"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
________________________________________________________________________________________

In [13]:
486//batch_size,

(15,)

In [14]:
model_facex.fit_generator(train_generator,
                      steps_per_epoch=len(train_generator),
                      epochs=25,
                      validation_data=validation_generator,
                      validation_steps=len(validation_generator),
) # 학습횟수를 25로 설정


Epoch 1/25
76/76 [==============================] - 1826s 24s/step - loss: 1.0018 - accuracy: 0.6257 - val_loss: 79.5356 - val_accuracy: 0.4715
Epoch 2/25
76/76 [==============================] - 1599s 21s/step - loss: 0.4583 - accuracy: 0.8135 - val_loss: 14708.8467 - val_accuracy: 0.4981
Epoch 3/25
76/76 [==============================] - 1618s 21s/step - loss: 0.1949 - accuracy: 0.9315 - val_loss: 423.6570 - val_accuracy: 0.5057
Epoch 4/25
76/76 [==============================] - 1623s 21s/step - loss: 0.1738 - accuracy: 0.9373 - val_loss: 341.6618 - val_accuracy: 0.5019
Epoch 5/25
76/76 [==============================] - 1640s 22s/step - loss: 0.1522 - accuracy: 0.9455 - val_loss: 4.5717 - val_accuracy: 0.5019
Epoch 6/25
76/76 [==============================] - 1641s 22s/step - loss: 0.1328 - accuracy: 0.9546 - val_loss: 1.2412 - val_accuracy: 0.9087
Epoch 7/25
76/76 [==============================] - 1643s 22s/step - loss: 0.1084 - accuracy: 0.9583 - val_loss: 8.5149e-08 - val_ac

In [15]:
model_facex.save("FINAL1.h5")